In [2]:
# 라이브러리 불러오기기
import pandas as pd
import matplotlib.pyplot as plt
import openai
import json
import os
from dotenv import load_dotenv

In [3]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, '..', 'data')
data_file_path = os.path.join(data_directory, 'description_complete2.csv')
df = pd.read_csv(data_file_path,index_col=0)

In [5]:
df['foreignn'] = df['foreignn'].apply(lambda x: '국내' if x in ['Domestic', 'domestic'] else x)
df['stay_type'] = df['stay_type'].apply(lambda x: '호텔' if x in ['Hotel', '해외 호텔','hotel','해외'] else x)
df.dropna(subset=['shipping_fee'],inplace=True)
df.reset_index(drop=True, inplace=True)
df['foreignn'] = df['foreignn'].fillna("국내")
df['stay_type'] = df['stay_type'].fillna("호텔")

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1912 entries, 0 to 1911
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   platform            1912 non-null   object 
 1   original_link       1912 non-null   object 
 2   post_time           1912 non-null   object 
 3   title               1912 non-null   object 
 4   view_count          1912 non-null   int64  
 5   like_count          1912 non-null   int64  
 6   price               1912 non-null   int64  
 7   images              1912 non-null   object 
 8   description         1912 non-null   object 
 9   category            1912 non-null   object 
 10  status              1912 non-null   object 
 11  seller_location     354 non-null    object 
 12  city_goo            136 non-null    object 
 13  citydong            341 non-null    object 
 14  description_1       1912 non-null   object 
 15  expiration_stdate   685 non-null    object 
 16  expira